# Adversarial Search: Playing Dots and Boxes


## Instructions

Total Points: Undegraduates 100, graduate students 110

Complete this notebook and submit it. The notebook needs to be a complete project report with your implementation, documentation including a short discussion of how your implementation works and your design choices, and experimental results (e.g., tables and charts with simulation results) with a short discussion of what they mean. Use the provided notebook cells and insert additional code and markdown cells as needed.

## Introduction

You will implement different versions of agents that play the game Dots and Boxes:

> "Dots and Boxes is a pencil-and-paper game for two players. The game starts with an empty grid of dots. Usually two players take turns adding a single horizontal or vertical line between two unjoined adjacent dots. A player who completes the fourth side of a 1x1 box earns one point and takes another turn. A point is typically recorded by placing a mark that identifies the player in the box, such as an initial. The game ends when no more lines can be placed. The winner is the player with the most points. The board may be of any size grid." (see [Dots and Boxes on Wikipedia](https://en.wikipedia.org/wiki/Dots_and_Boxes))

You can play Dots and Boxes [here](https://www.math.ucla.edu/~tom/Games/dots&boxes.html).

## Task 1: Defining the Search Problem [10 point]

Define the components of the search problem associated with this game:

* Initial state
* Actions
* Transition model
* Test for the terminal state
* Utility for terminal states

In [ ]:
import copy

# --- Define search problem components ---

def initial_state(rows=2, cols=2):
    """Tạo trạng thái ban đầu: chưa có cạnh nào được nối"""
    return {
        'rows': rows,
        'cols': cols,
        'edges': set(),              # các cạnh đã nối
        'scores': {'X': 0, 'O': 0},  # điểm của mỗi người chơi
        'turn': 'X'                  # người chơi hiện tại
    }

def all_edges(rows, cols):
    """Danh sách toàn bộ các cạnh hợp lệ"""
    edges = []
    for r in range(rows + 1):
        for c in range(cols):
            edges.append(((r, c), (r, c + 1)))   # cạnh ngang
    for r in range(rows):
        for c in range(cols + 1):
            edges.append(((r, c), (r + 1, c)))   # cạnh dọc
    return edges

def actions(state):
    """Trả về các cạnh chưa được nối"""
    rows, cols = state['rows'], state['cols']
    used = state['edges']
    all_e = set(all_edges(rows, cols))
    return list(all_e - used)

def result(state, action):
    """Thực hiện hành động (nối một cạnh) và trả về trạng thái mới"""
    new_state = copy.deepcopy(state)
    new_state['edges'].add(action)
    new_state['turn'] = 'O' if state['turn'] == 'X' else 'X'
    return new_state

def terminal(state):
    """Kiểm tra xem trò chơi đã kết thúc chưa"""
    rows, cols = state['rows'], state['cols']
    return len(state['edges']) == len(all_edges(rows, cols))

def utility(state, player='X'):
    """Tính điểm chênh lệch khi trò chơi kết thúc"""
    if not terminal(state):
        return None
    return state['scores'][player] - state['scores']['O' if player == 'X' else 'X']


How big is the state space? Give an estimate and explain it.

In [ ]:

R, C = 2, 2
E = 2*R*C + R + C
state_space = 2**E * 2
print(f"Tổng số cạnh: {E}, Ước lượng kích thước không gian trạng thái: {state_space:,}")


Tổng số cạnh: 12, Ước lượng kích thước không gian trạng thái: 8,192


How big is the game tree that minimax search will go through? Give an estimate and explain it.

In [ ]:
import math
game_tree_size = math.factorial(E)
print(f"Ước lượng kích thước cây trò chơi (trường hợp xấu nhất): {game_tree_size:e}")


Ước lượng kích thước cây trò chơi (trường hợp xấu nhất): 4.790016e+08


## Task 2: Game Environment and Random Agent [30 point]

You need to think about a data structure to represent the board meaning he placed lines and who finished what box. There are many options. Let's represent the board using a simple dictionary with components representing the board size, the lines and the boxes on the board.

**Important:** Everybody needs to use the same representation so we can let agents play against each other later.

In [ ]:
board = {
    'size': (4, 4),  ### number of rows and columns of dots
    'lines': dict(), ### keys are the set of drawn lines
    'boxes': dict    ### keys are the boxes and the value is the player who completed each box
}

def draw_line(board, orientation, row, col):
    """
    Place a line on an exiting board.

    Parameters
    ----------
    board: dict
        the board
    orientation: str
        either 'h' or 'v' for horizontal or vertical
    row, col: int
        index of the starting dot for the line (starting with 0)

    """

    if orientation not in ['h', 'v']:
        return False

    if row < 0 or col < 0:
        return False

    if row >= board['size'][0] + (orientation == 'v') or col >= board['size'][1] + (orientation == 'h'):
        return False

    if (orientation, row, col) in board['lines']:
        return False

    board["lines"][(orientation, row, col)] = True
    return True


print(draw_line(board, "h", 1, 1))
print(draw_line(board, "v", 1, 1))

# this should not work
print(draw_line(board, "h", 1, 1))

board

True
True
False


{'size': (4, 4),
 'lines': {('h', 1, 1): True, ('v', 1, 1): True},
 'boxes': dict}

Write code to display the board. **Bonus point: Post your visualization code with an example output to the discussion board. The best visualization will earn you bonus participation points in this class.**

In [ ]:
def display_board(board):
    """
    Hiển thị bàn chơi dưới dạng văn bản (ASCII).
    '—' : cạnh ngang đã nối
    '|' : cạnh dọc đã nối
    ' ' : cạnh chưa nối
    Các ô sẽ được đánh dấu theo người chơi nếu đã hoàn thành.
    """
    rows, cols = board["rows"], board["cols"]
    lines = board["lines"]
    boxes = board["boxes"]

    for r in range(rows + 1):
        # vẽ cạnh ngang
        line = ""
        for c in range(cols):
            line += "•"
            if ((r, c), (r, c + 1)) in lines or ((r, c + 1), (r, c)) in lines:
                line += "——"
            else:
                line += "  "
        line += "•"
        print(line)

        # vẽ cạnh dọc + ô giữa
        if r < rows:
            line = ""
            for c in range(cols + 1):
                if ((r, c), (r + 1, c)) in lines or ((r + 1, c), (r, c)) in lines:
                    line += "|"
                else:
                    line += " "
                # nếu có ô đã hoàn thành thì in ký hiệu người chơi
                if c < cols:
                    box_owner = boxes.get((r, c), " ")
                    line += " " + str(box_owner) + " "   # ép kiểu sang chuỗi
            print(line)
    print()


In [ ]:
def create_board(rows=2, cols=2):
    board = {
        "rows": rows,
        "cols": cols,
        "lines": set(),
        "boxes": {},
        "turn": "X",
        "scores": {"X": 0, "O": 0}
    }
    return board


In [ ]:
board = create_board(2, 2)
board["lines"].update([
    ((0,0),(0,1)), ((0,1),(0,2)), ((0,0),(1,0)), ((1,0),(1,1))
])
board["boxes"][(0,0)] = "X"
display_board(board)


•——•——•
| X      
•——•  •
         
•  •  •



Implement helper functions for:

* The transition model $result(s, a)$.
* The utility function $utility(s)$.
* Check for terminal states $terminal(s)$.
* A check for available actions in each state $actions(s)$.

__Notes:__
* Make sure that all these functions work with boards of different sizes (number of columns and rows as stored in the board).
* The result function updates the board and evaluates if the player closed a box and needs to store that information on the board. Add elements of the form `(row,col): player` to the board dictionary. `row` and `col` are the coordinates for the box and `player` is +1 or -1 representing the player. For example `(0,0): -1` means that the top-left box belongs to the other player.
* _Important:_ Remember that a player goes again after she completes a box!

In [ ]:
import copy

# --- Helper functions for the game environment ---

def actions(state):
    """
    Trả về danh sách các hành động hợp lệ (các cạnh chưa được nối).
    """
    rows, cols = state["rows"], state["cols"]
    all_edges = set()
    for r in range(rows + 1):
        for c in range(cols):
            all_edges.add(((r, c), (r, c + 1)))  # cạnh ngang
    for r in range(rows):
        for c in range(cols + 1):
            all_edges.add(((r, c), (r + 1, c)))  # cạnh dọc
    return list(all_edges - state["lines"])


def result(state, action):
    """
    Thực hiện hành động (nối 1 cạnh) và trả về trạng thái mới.
    - Nếu người chơi hoàn thành ô, cập nhật ô đó trong state["boxes"].
    - Người chơi được đi tiếp nếu hoàn thành ít nhất 1 ô.
    """
    new_state = copy.deepcopy(state)
    player = new_state["turn"]
    new_state["lines"].add(action)

    # Kiểm tra các ô được hoàn thành
    completed_box = False
    for r in range(new_state["rows"]):
        for c in range(new_state["cols"]):
            edges = {
                ((r, c), (r, c + 1)),         # cạnh trên
                ((r + 1, c), (r + 1, c + 1)), # cạnh dưới
                ((r, c), (r + 1, c)),         # cạnh trái
                ((r, c + 1), (r + 1, c + 1))  # cạnh phải
            }
            # nếu tất cả cạnh đều đã nối và ô chưa được ghi nhận
            if edges.issubset(new_state["lines"]) and (r, c) not in new_state["boxes"]:
                completed_box = True
                new_state["boxes"][(r, c)] = player
                new_state["scores"][player] = new_state["scores"].get(player, 0) + 1

    # Nếu không hoàn thành ô nào → đổi lượt
    if not completed_box:
        new_state["turn"] = -player  # đổi dấu (+1 ↔ -1)

    return new_state


def terminal(state):
    """
    Kiểm tra xem trò chơi đã kết thúc chưa:
    - Trò chơi kết thúc khi tất cả các cạnh đã được nối.
    """
    rows, cols = state["rows"], state["cols"]
    total_edges = (rows + 1) * cols + (cols + 1) * rows  # tổng số cạnh
    return len(state["lines"]) == total_edges


def utility(state):
    """
    Hàm đánh giá (utility):
    - Trả về hiệu điểm giữa người chơi +1 và -1.
    - Dùng khi trạng thái là terminal.
    """
    if not terminal(state):
        return None
    return state["scores"].get(+1, 0) - state["scores"].get(-1, 0)


In [ ]:
# Tạo bàn 2x2 cho test
state = {
    "rows": 2,
    "cols": 2,
    "lines": set(),
    "boxes": {},
    "scores": {+1: 0, -1: 0},
    "turn": +1
}

# Test thử
print("Các hành động hợp lệ ban đầu:", len(actions(state)))
a = actions(state)[0]
new_state = result(state, a)
print("Sau khi thực hiện hành động:", a)
print("Lượt chơi kế tiếp:", new_state["turn"])
print("Trò chơi kết thúc chưa?", terminal(new_state))


Các hành động hợp lệ ban đầu: 12
Sau khi thực hiện hành động: ((1, 1), (2, 1))
Lượt chơi kế tiếp: -1
Trò chơi kết thúc chưa? False


Implement an agent that plays randomly. Make sure the agent function receives as the percept the board and returns a valid action. Use an agent function definition with the following signature (arguments):

`def random_player(board, player = None): ...`

The argument `player` is used for agents that do not store what side they are playing. The value passed on by the environment should be 1 ot -1 for playerred and yellow, respectively.  See [Experiments section for tic-tac-toe](https://nbviewer.org/github/mhahsler/CS7320-AI/blob/master/Games/tictactoe_and_or_tree_search.ipynb#Experiments) for an example.

In [ ]:
import random

def random_player(board, player=None):
    possible_actions = actions(board)
    if not possible_actions:
        return None
    return random.choice(possible_actions)


In [ ]:
# Tạo bàn mới
board = {
    "rows": 2,
    "cols": 2,
    "lines": set(),
    "boxes": {},
    "scores": {+1: 0, -1: 0},
    "turn": +1
}

# Gọi random player
move = random_player(board, player=board["turn"])
print("Hành động ngẫu nhiên được chọn:", move)

# Áp dụng hành động đó
new_board = result(board, move)
print("Số cạnh sau khi đi:", len(new_board["lines"]))


Hành động ngẫu nhiên được chọn: ((1, 1), (1, 2))
Số cạnh sau khi đi: 1


Let two random agents play against each other 1000 times. Look at the [Experiments section for tic-tac-toe](https://nbviewer.org/github/mhahsler/CS7320-AI/blob/master/Games/tictactoe_and_or_tree_search.ipynb#Experiments) to see how the environment uses the agent functions to play against each other.

How often does each player win? Is the result expected?

In [ ]:
def play_random_vs_random(N=1000, rows=2, cols=2):
    """
    Cho hai người chơi ngẫu nhiên đấu với nhau N ván.
    Trả về số lần thắng của mỗi bên.
    """
    results = {+1: 0, -1: 0, 0: 0}  # 0 = hòa

    for _ in range(N):
        # Tạo bàn chơi mới
        board = {
            "rows": rows,
            "cols": cols,
            "lines": set(),
            "boxes": {},
            "scores": {+1: 0, -1: 0},
            "turn": +1
        }

        # Vòng lặp chơi cho đến khi kết thúc
        while not terminal(board):
            player = board["turn"]
            move = random_player(board, player)
            if move is None:
                break
            board = result(board, move)

        # Sau khi kết thúc, tính điểm
        score = utility(board)
        if score > 0:
            results[+1] += 1
        elif score < 0:
            results[-1] += 1
        else:
            results[0] += 1

    return results


# Chạy thử 1000 ván
results = play_random_vs_random(N=1000, rows=2, cols=2)
print("Kết quả sau 1000 ván đấu ngẫu nhiên:")
print(f"Người chơi +1 thắng: {results[+1]}")
print(f"Người chơi -1 thắng: {results[-1]}")
print(f"Hòa: {results[0]}")


Kết quả sau 1000 ván đấu ngẫu nhiên:
Người chơi +1 thắng: 420
Người chơi -1 thắng: 430
Hòa: 150


## Task 3: Minimax Search with Alpha-Beta Pruning [30 points]

### Implement the search starting.

Implement the search starting from a given board and specifying the player and put it into an agent function.
You can use code from the [tic-tac-toe example](https://nbviewer.org/github/mhahsler/CS7320-AI/blob/master/Games/tictactoe_alpha_beta_tree_search.ipynb).

__Notes:__
* Make sure that all your agent functions have a signature consistent with the random agent above.
* The search space for larger board may be too large. You can experiment with smaller boards.
* Tic-tac-toe does not have a rule where a player can go again if a box was completed. You need to adapt the tree search to reflect that rule.

In [ ]:
import math
import copy

# --- Hàm minimax có alpha-beta pruning ---
def minimax_value(board, player, alpha, beta, depth_limit=None):
    """
    Trả về giá trị tốt nhất của trạng thái board theo người chơi ban đầu (max_player).
    """
    if terminal(board):
        return utility(board)

    if depth_limit is not None and depth_limit <= 0:
        # heuristic = hiệu điểm hiện tại
        return board["scores"].get(+1, 0) - board["scores"].get(-1, 0)

    max_player = board["turn"]  # người đang chơi hiện tại
    actions_list = actions(board)

    if max_player == player:  # MAX node
        value = -math.inf
        for a in actions_list:
            new_board = result(board, a)
            # Nếu người chơi hoàn thành ô, họ sẽ được đi tiếp
            v = minimax_value(new_board, player, alpha, beta,
                              None if depth_limit is None else depth_limit - 1)
            value = max(value, v)
            alpha = max(alpha, value)
            if beta <= alpha:
                break  # cắt tỉa
        return value
    else:  # MIN node
        value = math.inf
        for a in actions_list:
            new_board = result(board, a)
            v = minimax_value(new_board, player, alpha, beta,
                              None if depth_limit is None else depth_limit - 1)
            value = min(value, v)
            beta = min(beta, value)
            if beta <= alpha:
                break
        return value


def minimax_decision(board, player, depth_limit=None):
    """
    Trả về hành động tốt nhất cho người chơi 'player' dựa trên minimax.
    """
    best_action = None
    best_value = -math.inf
    alpha, beta = -math.inf, math.inf

    for a in actions(board):
        new_board = result(board, a)
        value = minimax_value(new_board, player, alpha, beta,
                              None if depth_limit is None else depth_limit - 1)
        if value > best_value:
            best_value = value
            best_action = a
    return best_action


def minimax_player(board, player=None):
    return minimax_decision(board, player, depth_limit=4)  # có thể giảm/tăng depth nếu muốn


In [ ]:
# Tạo bàn 2x2
board = {
    "rows": 2,
    "cols": 2,
    "lines": set(),
    "boxes": {},
    "scores": {+1: 0, -1: 0},
    "turn": +1
}

# Gọi agent minimax
move = minimax_player(board, player=+1)
print("Nước đi minimax chọn:", move)


Nước đi minimax chọn: ((1, 1), (2, 1))


Experiment with some manually created boards (at least 3) to check if the agent spots winning opportunities. Discuss the results.

In [ ]:
# --- Thí nghiệm kiểm tra Minimax phát hiện cơ hội thắng ---

def test_manual_board(description, board):
    print(f"\n--- {description} ---")
    display_board(board)
    move = minimax_player(board, player=board["turn"])
    print(f"Người chơi {board['turn']} chọn hành động: {move}")
    new_board = result(board, move)
    display_board(new_board)
    print("Điểm sau khi đi:", new_board["scores"])


# ------------------- Thí nghiệm 1 -------------------
# Cơ hội hoàn thành ô bên trái trên cùng
board1 = {
    "rows": 2, "cols": 2,
    "lines": {
        ((0,0),(0,1)), ((0,1),(0,2)),
        ((0,0),(1,0)), ((1,0),(1,1))
    },
    "boxes": {},
    "scores": {+1: 0, -1: 0},
    "turn": +1
}

# ------------------- Thí nghiệm 2 -------------------
# Một ô sắp hoàn thành, người chơi -1 có thể giành điểm nếu thông minh
board2 = {
    "rows": 2, "cols": 2,
    "lines": {
        ((0,0),(0,1)), ((0,1),(0,2)),
        ((1,0),(1,1)), ((1,0),(2,0)), ((1,1),(2,1))
    },
    "boxes": {},
    "scores": {+1: 1, -1: 0},
    "turn": -1
}

# ------------------- Thí nghiệm 3 -------------------
# Cả hai ô gần hoàn tất, kiểm tra xem Minimax có chọn nước đi để tối đa điểm
board3 = {
    "rows": 2, "cols": 2,
    "lines": {
        ((0,0),(0,1)), ((0,1),(0,2)),
        ((0,0),(1,0)), ((1,0),(1,1)), ((1,1),(1,2)),
        ((1,0),(2,0)), ((1,1),(2,1))
    },
    "boxes": {(0,0): +1},
    "scores": {+1: 1, -1: 0},
    "turn": +1
}

# Thực hiện ba thí nghiệm
test_manual_board("Thí nghiệm 1: Người chơi +1 có thể hoàn thành ô", board1)
test_manual_board("Thí nghiệm 2: Người chơi -1 có thể ghi điểm", board2)
test_manual_board("Thí nghiệm 3: Người chơi +1 cần chọn nước tối ưu", board3)



--- Thí nghiệm 1: Người chơi +1 có thể hoàn thành ô ---
•——•——•
|        
•——•  •
         
•  •  •

Người chơi 1 chọn hành động: ((0, 1), (1, 1))
•——•——•
| 1 |    
•——•  •
         
•  •  •

Điểm sau khi đi: {1: 1, -1: 0}

--- Thí nghiệm 2: Người chơi -1 có thể ghi điểm ---
•——•——•
         
•——•  •
|   |    
•  •  •

Người chơi -1 chọn hành động: ((2, 1), (2, 2))
•——•——•
         
•——•  •
|   |    
•  •——•

Điểm sau khi đi: {1: 1, -1: 0}

--- Thí nghiệm 3: Người chơi +1 cần chọn nước tối ưu ---
•——•——•
| 1      
•——•——•
|   |    
•  •  •

Người chơi 1 chọn hành động: ((2, 0), (2, 1))
•——•——•
| 1      
•——•——•
| 1 |    
•——•  •

Điểm sau khi đi: {1: 2, -1: 0}


How long does it take to make a move? Start with a smaller board make the board larger. What is the largest board you can solve?

In [ ]:
import time

def measure_move_time(rows_list=[2, 3, 4], depth_limit=4):
    """
    Đo thời gian để Minimax chọn nước đi trên các kích thước bàn khác nhau.
    """
    times = {}

    for size in rows_list:
        board = {
            "rows": size,
            "cols": size,
            "lines": set(),
            "boxes": {},
            "scores": {+1: 0, -1: 0},
            "turn": +1
        }

        # Chọn ngẫu nhiên vài nước để bàn không quá trống
        # (giúp test thực tế hơn)
        for _ in range(min(3, size * size)):
            possible_moves = actions(board)
            if not possible_moves:
                break
            move = random.choice(possible_moves)
            board = result(board, move)

        # Đo thời gian ra quyết định
        start = time.time()
        _ = minimax_player(board, player=+1)
        end = time.time()
        elapsed = end - start
        times[size] = elapsed

        print(f"Bàn {size}x{size}: Minimax mất {elapsed:.3f} giây để chọn nước đi.")

    return times


# Thực hiện đo với các kích thước bàn khác nhau
results_time = measure_move_time(rows_list=[2, 3, 4])


Bàn 2x2: Minimax mất 0.058 giây để chọn nước đi.
Bàn 3x3: Minimax mất 0.944 giây để chọn nước đi.
Bàn 4x4: Minimax mất 4.117 giây để chọn nước đi.


### Move ordering

Starting the search with better moves will increase the efficiency of alpha-beta pruning. Describe and implement a simple move ordering strategy.

Make a table that shows how the ordering strategies influence the number of searched nodes and the search time?

In [ ]:
import math
import copy
import time

# Biến toàn cục đếm số nút duyệt
node_counter = 0

def ordered_actions(board):
    """
    Chiến lược sắp xếp nước đi đơn giản:
    - Ưu tiên các nước có khả năng hoàn thành ô (điểm thưởng ngay)
    - Các nước còn lại đứng sau.
    """
    moves = actions(board)
    scored_moves = []
    for move in moves:
        temp_board = result(board, move)
        # Ưu tiên nước nào giúp ghi điểm
        delta_score = (temp_board["scores"][+1] - board["scores"][+1]) + \
                      (temp_board["scores"][-1] - board["scores"][-1])
        scored_moves.append((move, delta_score))
    # Sắp xếp giảm dần theo điểm tiềm năng
    scored_moves.sort(key=lambda x: x[1], reverse=True)
    return [m[0] for m in scored_moves]


def minimax_value(board, player, alpha, beta, depth_limit=None, use_ordering=False):
    global node_counter
    node_counter += 1

    if terminal(board):
        return utility(board)

    if depth_limit is not None and depth_limit <= 0:
        return board["scores"][+1] - board["scores"][-1]

    current_player = board["turn"]
    # Sử dụng move ordering nếu bật
    moves = ordered_actions(board) if use_ordering else actions(board)

    if current_player == player:  # MAX
        value = -math.inf
        for move in moves:
            new_board = result(board, move)
            v = minimax_value(new_board, player, alpha, beta,
                              None if depth_limit is None else depth_limit - 1,
                              use_ordering)
            value = max(value, v)
            alpha = max(alpha, value)
            if beta <= alpha:
                break
        return value
    else:  # MIN
        value = math.inf
        for move in moves:
            new_board = result(board, move)
            v = minimax_value(new_board, player, alpha, beta,
                              None if depth_limit is None else depth_limit - 1,
                              use_ordering)
            value = min(value, v)
            beta = min(beta, value)
            if beta <= alpha:
                break
        return value


def minimax_decision(board, player, depth_limit=None, use_ordering=False):
    best_action = None
    best_value = -math.inf
    alpha, beta = -math.inf, math.inf

    for move in (ordered_actions(board) if use_ordering else actions(board)):
        new_board = result(board, move)
        value = minimax_value(new_board, player, alpha, beta,
                              None if depth_limit is None else depth_limit - 1,
                              use_ordering)
        if value > best_value:
            best_value = value
            best_action = move
    return best_action


def minimax_player(board, player=None, use_ordering=False):
    return minimax_decision(board, player, depth_limit=4, use_ordering=use_ordering)


### The first few moves

Start with an empty board. This is the worst case scenario for minimax search with alpha-beta pruning since it needs solve all possible games that can be played (minus some pruning) before making the decision. What can you do?

In [ ]:
def heuristic(board, player):
    # Ước lượng dựa trên số ô tiềm năng sắp hoàn thành
    score_diff = board["scores"][+1] - board["scores"][-1]
    near_boxes = sum(1 for (r,c) in all_possible_boxes(board)
                     if box_missing_lines(board, (r,c)) == 1)
    return score_diff + 0.1 * near_boxes * (1 if player == +1 else -1)


### Playtime

Let the Minimax Search agent play a random agent on a small board. Analyze wins, losses and draws.

In [ ]:
def play_match(agent1, agent2, N=50, rows=2, cols=2):
    results = {+1: 0, -1: 0, 0: 0}  # 0 = hòa

    for _ in range(N):
        board = {
            "rows": rows,
            "cols": cols,
            "lines": set(),
            "boxes": {},
            "scores": {+1: 0, -1: 0},
            "turn": +1  # luôn cho +1 đi trước
        }

        while not terminal(board):
            player = board["turn"]
            if player == +1:
                move = agent1(board, player)   # Minimax
            else:
                move = agent2(board, player)   # Random

            if move is None:
                break
            board = result(board, move)

        score = utility(board)

        if score > 0:
            results[+1] += 1
        elif score < 0:
            results[-1] += 1
        else:
            results[0] += 1

    return results


# Chạy thử 50 ván: Minimax (+1) vs Random (-1)
results = play_match(
    agent1=lambda b, p: minimax_player(b, p, use_ordering=True),
    agent2=random_player,
    N=50,
    rows=2,
    cols=2
)

print("Kết quả sau 50 ván:")
print(f"Minimax thắng: {results[+1]}")
print(f"Random thắng:  {results[-1]}")
print(f"Hòa:           {results[0]}")


Kết quả sau 50 ván:
Minimax thắng: 50
Random thắng:  0
Hòa:           0


## Task 4: Heuristic Alpha-Beta Tree Search [30 points]

### Heuristic evaluation function

Define and implement a heuristic evaluation function.

In [ ]:
def heuristic(board, player):
    """
    Hàm đánh giá heuristic cho Dots and Boxes.
    Trả về điểm số từ góc nhìn người chơi 'player'.
    """
    rows, cols = board["rows"], board["cols"]
    my_score = 0
    opp_score = 0
    nearly_complete = 0
    danger_boxes = 0

    # Đếm ô và số cạnh mỗi ô
    for r in range(rows):
        for c in range(cols):
            edges = {
                ((r, c), (r, c + 1)),
                ((r + 1, c), (r + 1, c + 1)),
                ((r, c), (r + 1, c)),
                ((r, c + 1), (r + 1, c + 1))
            }
            count_edges = sum([1 for e in edges if e in board["lines"]])

            # Nếu ô đã chiếm
            if (r, c) in board["boxes"]:
                if board["boxes"][(r, c)] == player:
                    my_score += 1
                else:
                    opp_score += 1

            # Nếu còn 1 cạnh nữa sẽ hoàn thành
            if count_edges == 3:
                nearly_complete += 1

            # Nếu đã có 2 cạnh → có nguy cơ bị đối thủ "bẫy"
            if count_edges == 2:
                danger_boxes += 1

    # Công thức heuristic (có thể tinh chỉnh)
    return (
        (my_score - opp_score) * 5 +     # trọng số mạnh
        nearly_complete * 2 -            # cơ hội ăn điểm
        danger_boxes                     # rủi ro cho nước đi sau
    )


### Cutting off search

Modify your Minimax Search with Alpha-Beta Pruning to cut off search at a specified depth and use the heuristic evaluation function. Experiment with different cutoff values.

In [ ]:
import math
import time
from functools import lru_cache

# Toàn cục để đo
node_counter = 0
cache_hits = 0

def board_key(board):
    lines_key = frozenset(board["lines"])
    boxes_key = tuple(sorted(board["boxes"].items()))
    turn = board["turn"]
    rows = board["rows"]
    cols = board["cols"]
    return (rows, cols, lines_key, boxes_key, turn)

def ordered_actions_simple(board):
    moves = actions(board)
    scored = []
    for m in moves:
        nb = result(board, m)   # result trả về board mới (đảm bảo deepcopy)
        # tổng điểm tăng (cả hai player) sau nước này
        delta = (nb["scores"].get(+1,0) + nb["scores"].get(-1,0)) - (board["scores"].get(+1,0) + board["scores"].get(-1,0))
        scored.append((m, delta))
    scored.sort(key=lambda x: x[1], reverse=True)
    return [m for m,_ in scored]

def minimax_ab(board, max_player, depth_limit=None, alpha=-math.inf, beta=math.inf, use_ordering=False, transposition=True):
    global node_counter, cache_hits
    node_counter += 1

    # terminal?
    if terminal(board):
        return utility(board)

    # cutoff
    if depth_limit is not None and depth_limit <= 0:
        return heuristic(board, max_player)

    # memoization key
    key = None
    if transposition:
        try:
            key = board_key(board)
        except Exception:
            key = None

    if key is not None:
        if "_tt" not in globals():
            global _tt; _tt = {}
        entry = _tt.get((key, depth_limit, max_player))
        if entry is not None:
            cache_hits += 1
            return entry

    current_player = board["turn"]

    moves = ordered_actions_simple(board) if use_ordering else actions(board)

    if current_player == max_player:
        value = -math.inf
        for m in moves:
            nb = result(board, m)
            v = minimax_ab(nb, max_player, None if depth_limit is None else depth_limit-1,
                           alpha, beta, use_ordering, transposition)
            if v > value:
                value = v
            alpha = max(alpha, value)
            if beta <= alpha:
                break
    else:
        value = math.inf
        for m in moves:
            nb = result(board, m)
            v = minimax_ab(nb, max_player, None if depth_limit is None else depth_limit-1,
                           alpha, beta, use_ordering, transposition)
            if v < value:
                value = v
            beta = min(beta, value)
            if beta <= alpha:
                break

    if key is not None:
        _tt[(key, depth_limit, max_player)] = value

    return value

def minimax_decision(board, player, depth_limit=None, use_ordering=False, transposition=True):
    """
    Trả về best action cho 'player' (player = max_player).
    """
    best_action = None
    best_value = -math.inf
    alpha = -math.inf
    beta = math.inf

    moves = ordered_actions_simple(board) if use_ordering else actions(board)
    for m in moves:
        nb = result(board, m)
        v = minimax_ab(nb, player, None if depth_limit is None else depth_limit-1,
                       alpha, beta, use_ordering, transposition)
        if v > best_value:
            best_value = v
            best_action = m
        alpha = max(alpha, best_value)
        if beta <= alpha:
            break
    return best_action

def minimax_player_with_params(board, player=None, depth_limit=3, use_ordering=True, transposition=True):
    return minimax_decision(board, player, depth_limit=depth_limit, use_ordering=use_ordering, transposition=transposition)


In [ ]:
import time

def measure(board_setup_fn, depth, use_ordering=True, transposition=True):
    global node_counter, cache_hits, _tt
    node_counter = 0
    cache_hits = 0
    _tt = {}
    board = board_setup_fn()
    t0 = time.time()
    move = minimax_player_with_params(board, player=board["turn"], depth_limit=depth, use_ordering=use_ordering, transposition=transposition)
    elapsed = time.time() - t0
    return {"move": move, "time": elapsed, "nodes": node_counter, "cache_hits": cache_hits}


def make_empty_2():
    return {"rows":2,"cols":2,"lines":set(),"boxes":{},"scores":{+1:0,-1:0},"turn":+1}

print(measure(make_empty_2, depth=3))
print(measure(make_empty_2, depth=4))


{'move': ((2, 1), (2, 2)), 'time': 0.019321680068969727, 'nodes': 185, 'cache_hits': 17}
{'move': ((1, 0), (2, 0)), 'time': 0.029041767120361328, 'nodes': 263, 'cache_hits': 23}


How many nodes are searched and how long does it take to make a move? Start with a smaller board with 4 columns and make the board larger by adding columns.

In [ ]:
import time

def make_empty_board(rows, cols):
    return {
        "rows": rows,
        "cols": cols,
        "lines": set(),
        "boxes": {},
        "scores": {+1:0, -1:0},
        "turn": +1
    }

def experiment_columns(column_list=[4,5,6,7], depth_limit=3):
    global node_counter, cache_hits, _tt

    results = []
    print(f"{'Cols':<6} | {'Time (s)':<12} | {'Nodes':<12} | {'Cache Hits'}")
    print("-"*55)

    for c in column_list:
        # reset boards + counters
        board = make_empty_board(2, c)
        node_counter = 0
        cache_hits = 0
        _tt = {}

        start = time.time()
        move = minimax_player_with_params(board, player=+1, depth_limit=depth_limit, use_ordering=True)
        elapsed = time.time() - start

        results.append((c, elapsed, node_counter, cache_hits))

        print(f"{c:<6} | {elapsed:<12.4f} | {node_counter:<12} | {cache_hits}")

    return results

# chạy thử
experiment_columns([4,5,6,7,8], depth_limit=3)


Cols   | Time (s)     | Nodes        | Cache Hits
-------------------------------------------------------
4      | 0.0455       | 289          | 21
5      | 0.0595       | 359          | 26
6      | 0.0818       | 431          | 31
7      | 0.1187       | 496          | 36
8      | 0.1596       | 567          | 41


[(4, 0.045513153076171875, 289, 21),
 (5, 0.05945754051208496, 359, 26),
 (6, 0.08184552192687988, 431, 31),
 (7, 0.1186819076538086, 496, 36),
 (8, 0.15959668159484863, 567, 41)]

In [ ]:
import pandas as pd

res = experiment_columns([4,5,6,7,8], depth_limit=3)
df = pd.DataFrame(res, columns=["Columns", "Time (s)", "Nodes", "Cache Hits"])
df


Cols   | Time (s)     | Nodes        | Cache Hits
-------------------------------------------------------
4      | 0.0411       | 289          | 21
5      | 0.0607       | 359          | 26
6      | 0.0874       | 431          | 31
7      | 0.1112       | 496          | 36
8      | 0.1577       | 567          | 41


,Columns,Time (s),Nodes,Cache Hits
0,4,0.041115,289,21
1,5,0.060673,359,26
2,6,0.087380,431,31
3,7,0.111166,496,36
4,8,0.157663,567,41


### Playtime

Let two heuristic search agents (different cutoff depth, different heuristic evaluation function) compete against each other on a reasonably sized board. Since there is no randomness, you only need to let them play once.

In [ ]:
# ============= Heuristic 2 =============
def heuristic2(board, player):
    """
    Heuristic khác:
    - ưu tiên ô gần hoàn thành (3 cạnh)
    - tránh trạng thái đối thủ có thể hoàn thành ô
    """
    score = board["scores"].get(player, 0) - board["scores"].get(-player, 0)

    danger = 0
    opportunity = 0

    for r in range(board["rows"]):
        for c in range(board["cols"]):
            edges = 0
            # check 4 cạnh của ô (r, c)
            if ((r,c),(r,c+1)) in board["lines"]: edges += 1
            if ((r+1,c),(r+1,c+1)) in board["lines"]: edges += 1
            if ((r,c),(r+1,c)) in board["lines"]: edges += 1
            if ((r,c+1),(r+1,c+1)) in board["lines"]: edges += 1

            if edges == 3:
                opportunity += 2
            if edges == 2:
                danger += 1

    return score + opportunity - danger


# ============= Agent A / Agent B =============
def agent_A(board, player=None):
    # dùng heuristic mặc định của Minimax, depth=3
    return minimax_decision(board, player,
                            depth_limit=3,
                            use_ordering=True)


def agent_B(board, player=None):
    # Override heuristic tạm thời => dùng heuristic2
    global heuristic
    saved_h = heuristic
    heuristic = heuristic2

    move = minimax_decision(board, player,
                            depth_limit=5,
                            use_ordering=True)

    # khôi phục heuristic mặc định
    heuristic = saved_h
    return move


# ============= Môi trường đấu 2 agent =============
def play_game(agent1, agent2, rows=2, cols=5):
    # tạo board ban đầu rỗng
    board = {
        "rows": rows,
        "cols": cols,
        "lines": set(),
        "boxes": {},
        "scores": {+1:0, -1:0},
        "turn": +1
    }

    # loop đến khi terminal
    while not terminal(board):
        if board["turn"] == +1:
            move = agent1(board, +1)
        else:
            move = agent2(board, -1)
        board = result(board, move)

    return board


# ============= Chạy trận đấu =============
final_board = play_game(agent_A, agent_B, rows=2, cols=5)
print("Final scores:", final_board["scores"])

p1 = final_board["scores"][+1]
p2 = final_board["scores"][-1]

print("\n--- KẾT QUẢ ---")
print("Player +1 (Agent A):", p1)
print("Player -1 (Agent B):", p2)

if p1 > p2:
    print("==> Agent A thắng!")
elif p1 < p2:
    print("==> Agent B thắng!")
else:
    print("==> Hòa!")


Final scores: {1: 8, -1: 2}

--- KẾT QUẢ ---
Player +1 (Agent A): 8
Player -1 (Agent B): 2
==> Agent A thắng!


## Tournament task [+1 to 5% bonus on your course grade; will be assigned separately]

Find another student and let your best agent play against the other student's best player. You are allowed to use any improvements you like as long as you code it yourself. We will set up a class tournament on Canvas. This tournament will continue after the submission deadline.

## Graduate student advanced task: Pure Monte Carlo Search and Best First Move [10 point]

__Undergraduate students:__ This is a bonus task you can attempt if you like [+5 Bonus point].

### Pure Monte Carlo Search

Implement Pure Monte Carlo Search (see [tic-tac-toe-example](https://nbviewer.org/github/mhahsler/CS7320-AI/blob/master/Games/tictactoe_pure_monte_carlo_search.ipynb)) and investigate how this search performs on the test boards that you have used above.

In [ ]:
# Your code/ answer goes here.

### Best First Move

How would you determine what the best first move for a standard board ($5 \times 5$) is? You can use Pure Monte Carlo Search or any algorithms that you have implemented above.

In [ ]:
# Your code/ answer goes here.